# Confronto: insertion-sort vs counting-sort

Notebook pratico che confronta le prestazioni di insertion-sort e counting-sort su array di interi. Le celle alternano brevi spiegazioni (markdown) e codice eseguibile.

## Import e impostazioni
Impostiamo l'ambiente: seed riproducibile e stile per i grafici.

In [1]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt
RNG = np.random.default_rng(42)
plt.style.use('seaborn-v0_8-darkgrid')

## Insertion sort
Implementazione semplice (opera su una copia). Complessità: O(n^2).

In [2]:
def insertion_sort(a):
    """Ordina `a` e ritorna una nuova lista ordinata.
    Non modifica l'input."""
    if isinstance(a, np.ndarray):
        arr = a.tolist()
    else:
        arr = list(a)
    n = len(arr)
    for i in range(1, n):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > key:
            arr[j+1] = arr[j]
            j -= 1
        arr[j+1] = key
    return arr

### Teoria: insertion-sort
- **Complessità temporale**:
  - Caso migliore: O(n) quando l'array è già ordinato (solo n-1 confronti, nessuno scambio)
  - Caso medio: O(n²) con circa n²/4 confronti e n²/4 scambi in media
  - Caso peggiore: O(n²) con n²/2 confronti e n²/2 scambi (array in ordine inverso)
- **Complessità spaziale**:
  - O(1) addizionale per la versione in-place (la versione del notebook copia l'input quindi usa O(n))
  - Non richiede strutture dati ausiliarie oltre all'array stesso
- **Stabilità**: È un algoritmo stabile, mantiene l'ordine relativo degli elementi con chiavi uguali
- **Caratteristiche**:
  - Adattivo: prestazioni migliori su array parzialmente ordinati
  - Online: può ordinare un flusso di dati man mano che arrivano
  - Efficiente su piccoli dataset (overhead costante basso)
  - Usato come base in algoritmi ibridi (es. timsort di Python/Java)
- **Risorse computazionali**:
  - Basso overhead: nessuna chiamata ricorsiva
  - Localizzazione della memoria: buon uso della cache per piccoli array

Esempio rapido di `insertion_sort`.

In [3]:
sample = RNG.integers(0, 50, size=12)
print('sample:', sample)
print('insertion_sort result:', insertion_sort(sample))

sample: [ 4 38 32 21 21 42  4 34 10  4 26 48]
insertion_sort result: [4, 4, 4, 10, 21, 21, 26, 32, 34, 38, 42, 48]


## Counting sort
Counting sort per interi non negativi. Complessità: O(n + k).

In [8]:
def counting_sort(a):
    """Counting sort classico che calcola internamente il valore massimo.
    - Accetta liste o numpy array.
    - Supporta solo interi non negativi.
    - Solleva ValueError per valori negativi e MemoryError se k (max+1) è troppo grande.
    """
    if isinstance(a, np.ndarray):
        arr = a.tolist()
    else:
        arr = list(a)
    if not arr:
        return []
    # Verifica che non ci siano valori negativi
    if min(arr) < 0:
        raise ValueError('Counting sort: supporta solo interi non negativi')
    # calcolo diretto del massimo
    max_value = max(arr)
    k = max_value + 1
    # limite di sicurezza per evitare OOM accidentali
    MAX_K_LIMIT = 50_000_000
    if k > MAX_K_LIMIT:
        raise MemoryError(f'k troppo grande per il counting sort: k={k} > {MAX_K_LIMIT}')
    counts = [0] * k
    for v in arr:
        counts[v] += 1
    out = []
    for val, c in enumerate(counts):
        if c:
            out.extend([val] * c)
    return out

### Teoria: counting-sort e varianti
- **Complessità temporale**:
  - O(n + k) in tutti i casi (migliore, medio e peggiore)
  - Non dipende dall'ordine iniziale degli elementi
  - La complessità lineare O(n) è raggiungibile solo quando k = O(n)
- **Complessità spaziale**:
  - O(n + k) totale: O(n) per l'output e O(k) per l'array di conteggio
  - Il fattore dominante è max(n, k)
  - Per k >> n diventa inefficiente in termini di memoria
- **Caratteristiche**:
  - Non basato su confronti (comparison-free)
  - Stabile nella versione con array cumulativo (non quella implementata)
  - Limitato a interi o valori mappabili a interi in un range limitato
- **Trade-off**:
  - Ottimo quando k è piccolo rispetto a n
  - Inefficiente quando k >> n (per memoria e tempo)
- **Varianti**:
  - Counting sort generalizzato per range (min, max) qualsiasi
  - Versione stabile con array cumulativo per preservare l'ordine di chiavi uguali
  - Implementazioni ottimizzate con linguaggi compilati o librerie specializzate
- **Risorse computazionali**:
  - Utilizza la memoria in modo prevedibile ma potenzialmente significativo
  - Accesso sequenziale agli array favorisce la cache locality
  - Efficiente in termini di branch prediction (pochi branch condizionali)

Esempio rapido di `counting_sort` (ora senza passare il valore massimo).

In [5]:
print('counting_sort result (sample):', counting_sort(sample))

counting_sort result (sample): [4, 4, 4, 10, 21, 21, 26, 32, 34, 38, 42, 48]


## Test di correttezza
Verifica su casi semplici e edge case.

In [6]:
def is_sorted(a):
    return all(a[i] <= a[i+1] for i in range(len(a)-1))

# più test, includendo casi con ripetizioni e range piccoli/grandi
tests = [[], [1], [2,1], [5,3,8,1,2,7], list(RNG.integers(0,50,size=20)), [0,0,0,0], list(range(10,0,-1))]
for t in tests:
    res_i = insertion_sort(t)
    res_c = counting_sort(t)
    assert is_sorted(res_i)
    assert is_sorted(res_c)
    assert sorted(list(t)) == res_i == res_c
print('Tutti i test di correttezza passati.')

Tutti i test di correttezza passati.


### Teoria: correttezza e limitazioni pratiche
- **Correttezza algoritmica**:
  - Insertion sort: corretto per definizione, ogni elemento è inserito nella posizione appropriata
  - Counting sort: corretto se l'intervallo dei valori è noto e gestibile in memoria
- **Invarianti garantite**:
  - Insertion sort mantiene l'invariante che il sottoarray [0..i-1] è sempre ordinato
  - Counting sort garantisce conteggi corretti per ogni valore nel range [0..k-1]
- **Proprietà di stabilità**:
  - Insertion sort è naturalmente stabile (preserva l'ordine relativo di elementi uguali)
  - La versione di counting sort implementata qui non è stabile, ma esistono versioni stabili
- **Limitazioni pratiche**:
  - Python: overhead dell'interprete influenza le prestazioni di entrambi gli algoritmi
  - Conversione numpy→list: aggiunge costi in tempo e memoria significativi
  - Implementazioni ibride o vettorizzate (usando NumPy) potrebbero offrire vantaggi significativi
  - Ottimizzazioni come JIT (Just-In-Time) compilation (es. Numba) potrebbero ridurre l'overhead

## Benchmark: tempo e memoria
Misuriamo tempo medio (con deviazione) e picco di memoria Python allocata (usando tracemalloc).
Eseguiamo test su diverse combinazioni di n (dimensione) e k (massimo valore possibile) e su due distribuzioni: uniforme e clusterizzata.

In [ ]:
import tracemalloc
import math
from statistics import mean, stdev


def time_mem_function(func, args=(), repeat=5):
    times = []
    mem_peaks = []
    for _ in range(repeat):
        tracemalloc.start()
        t0 = time.perf_counter()
        try:
            func(*args)
        finally:
            t1 = time.perf_counter()
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
        times.append(t1 - t0)
        mem_peaks.append(peak)
    mean_t = float(mean(times))
    std_t = float(stdev(times)) if len(times) > 1 else 0.0
    mean_m = float(mean(mem_peaks))
    std_m = float(stdev(mem_peaks)) if len(mem_peaks) > 1 else 0.0
    return (mean_t, std_t, mean_m, std_m)

# warm-up
warm = RNG.integers(0, 100, size=200)
_ = insertion_sort(warm)
_ = counting_sort(warm)
print('Warm-up eseguito.')

# definizioni dei test (estesi)
n_values = [100, 300, 1000, 3000]
k_values = [10, 100, 1000, 10000]
repeat = 5

results = []  # lista di dicts: {dist, n, k, algo, time_mean, time_std, mem_mean, mem_std}

for k in k_values:
    for n in n_values:
        # distribuzione uniforme su [0, k]
        data_uniform = RNG.integers(0, k+1, size=n)
        # distribuzione cluster (molti valori piccoli, pochi grandi)
        small_part = RNG.integers(0, max(1, k//10)+1, size=int(n*0.9))
        big_part = RNG.integers(0, k+1, size=int(n*0.1))
        data_cluster = np.concatenate([small_part, big_part])
        for dist_name, data in [('uniform', data_uniform), ('cluster', data_cluster)]:
            # insertion sort
            mean_i, std_i, mean_mi, std_mi = time_mem_function(insertion_sort, (data,), repeat=repeat)
            results.append({'dist': dist_name, 'n': n, 'k': k, 'algo': 'insertion', 'time_mean': mean_i, 'time_std': std_i, 'mem_mean': mean_mi, 'mem_std': std_mi})
            # counting sort (proteggiamo da MemoryError)
            try:
                mean_c, std_c, mean_mc, std_mc = time_mem_function(counting_sort, (data,), repeat=repeat)
            except MemoryError as me:
                print(f'MemoryError for counting_sort with k={k}, n={n}:', me)
                mean_c = std_c = mean_mc = std_mc = math.nan
            results.append({'dist': dist_name, 'n': n, 'k': k, 'algo': 'counting', 'time_mean': mean_c, 'time_std': std_c, 'mem_mean': mean_mc, 'mem_std': std_mc})
            print(f'k={k} n={n} dist={dist_name} -> insertion {mean_i:.6f}s (±{std_i:.6f}), mem={mean_mi:.0f}B; counting {mean_c:.6f}s (±{std_c:.6f}), mem={mean_mc:.0f}B')

# grafici: per ogni k e distribuzione, tempo e memoria in subplot affiancati
from IPython.display import Image, display

for k in k_values:
    for dist in ['uniform', 'cluster']:
        subset = [r for r in results if r['k']==k and r['dist']==dist]
        ns = sorted(set(r['n'] for r in subset))
        ins = [r for r in subset if r['algo']=='insertion']
        cnt = [r for r in subset if r['algo']=='counting']
        if not ins or not cnt:
            continue
        ins_times = [next(r['time_mean'] for r in ins if r['n']==n) for n in ns]
        cnt_times = [next(r['time_mean'] for r in cnt if r['n']==n) for n in ns]
        ins_mem = [next(r['mem_mean'] for r in ins if r['n']==n) for n in ns]
        cnt_mem = [next(r['mem_mean'] for r in cnt if r['n']==n) for n in ns]

        plt.figure(figsize=(10,4))
        plt.subplot(1,2,1)
        plt.plot(ns, ins_times, label='insertion-sort', marker='o')
        plt.plot(ns, cnt_times, label=f'counting-sort (k={k})', marker='s')
        plt.xlabel('n')
        plt.ylabel('Tempo medio (s)')
        plt.title(f'Tempo - k={k} dist={dist}')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)

        plt.subplot(1,2,2)
        plt.plot(ns, ins_mem, label='insertion-sort mem', marker='o')
        plt.plot(ns, cnt_mem, label='counting-sort mem', marker='s')
        plt.xlabel('n')
        plt.ylabel('Memoria picco (bytes)')
        plt.title(f'Memoria (picco) - k={k} dist={dist}')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)

        # Crea directory se non esiste
        import os
        os.makedirs('noteBookPng', exist_ok=True)

        # Salva sia nella directory principale che nella sottodirectory noteBookPng
        fname_subdir = f'noteBookPng/compare_insertion_counting_k{k}_dist_{dist}.png'

        plt.tight_layout()
        plt.savefig(fname_subdir)

        print(f'Salvato {fname_subdir}')
        plt.close()

        # Visualizza l'immagine appena creata
        display(Image(filename=fname_main))





Warm-up eseguito.
k=10 n=100 dist=uniform -> insertion 0.000133s (±0.000019), mem=888B; counting 0.000034s (±0.000007), mem=1880B
k=10 n=100 dist=cluster -> insertion 0.000063s (±0.000002), mem=888B; counting 0.000027s (±0.000007), mem=2152B
k=10 n=300 dist=uniform -> insertion 0.003363s (±0.000988), mem=6754B; counting 0.000062s (±0.000027), mem=5304B
k=10 n=300 dist=cluster -> insertion 0.002625s (±0.000705), mem=2564B; counting 0.000070s (±0.000014), mem=5912B
k=10 n=1000 dist=uniform -> insertion 0.221108s (±0.008052), mem=9008B; counting 0.000080s (±0.000009), mem=17864B
k=10 n=1000 dist=cluster -> insertion 0.111305s (±0.003903), mem=9102B; counting 0.000352s (±0.000030), mem=19240B
k=10 n=3000 dist=uniform -> insertion 2.676060s (±0.179725), mem=25034B; counting 0.000337s (±0.000013), mem=51480B
k=10 n=3000 dist=cluster -> insertion 1.307589s (±0.049971), mem=25034B; counting 0.001633s (±0.000026), mem=57640B
k=100 n=100 dist=uniform -> insertion 0.000097s (±0.000002), mem=888B;

## Visualizzazione grafici

### Confronto per k=10
![Confronto insertion vs counting k=10 uniform](noteBookPng/compare_insertion_counting_k10_dist_uniform.png)
![Confronto insertion vs counting k=10 cluster](noteBookPng/compare_insertion_counting_k10_dist_cluster.png)

### Confronto per k=100
![Confronto insertion vs counting k=100 uniform](noteBookPng/compare_insertion_counting_k100_dist_uniform.png)
![Confronto insertion vs counting k=100 cluster](noteBookPng/compare_insertion_counting_k100_dist_cluster.png)

### Confronto per k=1000
![Confronto insertion vs counting k=1000 uniform](noteBookPng/compare_insertion_counting_k1000_dist_uniform.png)
![Confronto insertion vs counting k=1000 cluster](noteBookPng/compare_insertion_counting_k1000_dist_cluster.png)

### Confronto per k=10000
![Confronto insertion vs counting k=10000 uniform](noteBookPng/compare_insertion_counting_k10000_dist_uniform.png)
![Confronto insertion vs counting k=10000 cluster](noteBookPng/compare_insertion_counting_k10000_dist_cluster.png)

## Teoria: benchmarking e misurazioni
- **Analisi della memoria con tracemalloc**:
  - tracemalloc misura le allocazioni del runtime Python (heap) e il picco di memoria allocata dalle strutture Python durante l'esecuzione della funzione
  - NON misura il consumo di memoria a livello di processo (RSS), ma solo il memory footprint gestito da Python
  - Le misure catturano sia la memoria occupata dalle strutture dati degli algoritmi che l'overhead del runtime Python
  - I conteggi includono tutti gli oggetti creati durante l'esecuzione degli algoritmi, come liste temporanee e altri oggetti allocati dinamicamente

- **Perché il warmup**:
  - Negli ambienti Python, la prima esecuzione di una funzione è spesso più lenta (compilazione JIT, cache, ottimizzazioni interne)
  - Il warmup prepara l'interprete e le strutture dati interne per misurazioni più stabili
  - Neutralizza effetti come: inizializzazione lazy, compilazione JIT (per PyPy), ottimizzazioni del GC, ecc.
  - In questo esperimento, eseguiamo insertion_sort e counting_sort su un array di test prima di iniziare le misurazioni reali

- **Metodologia di misurazione**:
  - Ripetizione multipla: ogni misura è ripetuta 5 volte per ottenere valori statisticamente significativi
  - Calcolo di media e deviazione standard: per valutare sia le prestazioni tipiche che la loro variabilità
  - Misurazioni separate per tempo e memoria: entrambe le risorse sono critiche nella valutazione degli algoritmi

- **Considerazioni sull'overhead**:
  - tracemalloc e strumenti di profiling introducono un piccolo overhead di misura
  - Confronti puramente temporali dovrebbero essere fatti sia con che senza misurazione della memoria
  - I valori assoluti possono variare tra diverse esecuzioni, ma le tendenze relative sono generalmente consistenti

## Raccomandazioni pratiche
- Proteggere la versione di counting sort con una soglia di sicurezza su k (per evitare OOM accidentali). Qui usiamo `MAX_K_LIMIT` come valore prudente; puoi adattarlo in base alla RAM disponibile.
- Per dataset grandi con valori in intervalli ampi, preferire algoritmi O(n log n) come quicksort o mergesort.
- Se servono dati stabili con payload, usare la versione stabile di counting sort o alternative come radix sort.

## Conclusioni: analisi comparativa e risorse utilizzate

### Complessità teorica
- **Insertion sort**:
  - Tempo: O(n²) in generale, O(n) nel caso migliore (array già ordinato)
  - Spazio: O(1) addizionale per la versione in-place

- **Counting sort**:
  - Tempo: O(n + k) in tutti i casi
  - Spazio: O(n + k) totale, con O(k) per l'array di conteggio

### Osservazioni empiriche
- **Risorse temporali**:
  - L'insertion sort diventa rapidamente inefficiente al crescere di n (confermando la complessità quadratica)
  - Il counting sort mantiene prestazioni lineari quando k è contenuto
  - Per k molto grande rispetto a n, il counting sort perde il suo vantaggio competitivo

- **Risorse di memoria**:
  - L'insertion sort ha un consumo di memoria proporzionale alla dimensione dell'input (O(n))
  - Il counting sort richiede memoria aggiuntiva significativa per l'array dei conteggi (O(k))
  - Per valori di k molto grandi (k >> n), il counting sort può diventare proibitivo in termini di memoria

- **Considerazioni sulla distribuzione dei dati**:
  - La distribuzione dei valori nell'array influenza poco l'insertion sort
  - Il counting sort è completamente insensibile alla distribuzione dell'input
  - Su distribuzioni cluster, le prestazioni relative non cambiano significativamente

### Applicazioni pratiche
- **Quando usare insertion sort**:
  - Array piccoli (n < 10-20)
  - Array quasi ordinati
  - Quando la memoria è una risorsa critica
  - Come parte di algoritmi ibridi (es. per piccoli sottoinsiemi)

- **Quando usare counting sort**:
  - Range di valori limitato e noto (k = O(n))
  - Grandi dataset con valori ripetuti in un intervallo ristretto
  - Quando la velocità è prioritaria rispetto alla memoria
  - Come componente di algoritmi più sofisticati (es. radix sort)

### Ottimizzazioni possibili
- Per insertion sort: implementazioni cache-friendly, branch prediction-friendly
- Per counting sort: range compression, varianti con ottimizzazioni di memoria, integrazione con librerie vettoriali

In conclusione, la scelta tra insertion sort e counting sort dipende criticamente dal rapporto tra n (dimensione dell'input) e k (range dei valori), oltre che dalle risorse di sistema disponibili e dai requisiti dell'applicazione.
